# ```tagger.py``` : Tagging sentences using the default tagger
```tagger.py``` writes part-of-speech tags for sentences using the default tagger (trained on WSJ).

*Note* : Since it uses ```default_tagger.pk```, ```wsj_train.txt```, and ```wsj_test.txt``` to optimize.
If it does not exist in ```proof-corpus/```, it will make these files.

## Input
- --files, -f
    - tsv files of sentences 

## Other arguments
- --cores, -p
    - Number of cores to use. 
    - Defaults to 4.
- --test, -t
    - Prints results (instead of writing to file)
- --raw, -r
    - Do not correct bracket or alias tags.
        - ex : (, [, CASE, CITE.

## Output
- --output, -o
    - Path of (a single) txt file of all the input sentences with their tag 
    - Example output in ```tagged_sents/```

## Example usage

In [18]:
!nohup python3 tagger.py --files ../../2023-01-04/proof-corpus/sent01.tsv --cores 25 --output tagged_sents/tagged_sentences_test.txt

nohup: ignoring input and appending output to 'nohup.out'


# ```extract_sents.py``` : Create word bins for optimal tagger
```extract_sents.py``` creates word bins for optimal tagger experiment. 
    Specifically, it extracts sentences that begin with words in word_file from tagged sentences.

## Input
- --file, -f
    - Tagged sentences. 
    - Saved in ```tagged_sents/```.
    - Output of ```tagger.py```.

- --word_file, -wf
    - Path of txt file containing word list to make bins.
        - ex : ```optimal_tagger_extra/word_bin_list.txt```)
        *OR*
- --word, -w
    - the word itself as a string (eg. Note)

*Note* : The word must be capitalized for both --word and in --word_list

## Other arguments
- --unique, -u
    - Flag to store unique sentences separately.
    - Saved in ```word_bins/unique```.
    - If this script is being run in order to make ```training/testing_sets``` using ```make_test_train.py```, add the -u flag as ```make_test_train.py``` uses unique sentences (sentences in ```word_bins/unique```) by default.

- --cores, -p
    - Number of cores to use. 
    - Defaults to 4.
    
- --extension, -e
    - Custom extension for filename. 
    - Adds string to the end of filename.

## Output
- txt files of sentences starting with specified word (in ```word_bins/``` or ```word_bins/unique/```)
    - Default :
        - Output is saved in ```word_bins/``` or ```word_bins/unique/``` depending on whether the -u flag is used.
        - The file name is automatically formatted to be the word (by which we're creating the word bin).

            > ```file_name = "word_bins/" + word + args.extension + ".txt"```

            > ```file_name_unique = "word_bins/unique/" + word + args.extension + ".txt"```

    - --output, -o
        - Path of txt file to write sentences to.
                            
    - --unique_output, -uo
        - Path of txt file to write unique sentences to.

## Example usage

In [19]:
!nohup python3 extract_sents.py --file tagged_sents/tagged_sentences_test.txt --word_file word_bin_list.txt --cores 20 --unique

nohup: ignoring input and appending output to 'nohup.out'


# ```make_test_train.py``` : Make testing and training bins
```make_test_train.py``` makes training and testing sets from word bins for optimal tagger experiment.

## Input
- --word_list 
    - Filename of txt file containing the list of words that should be used as the ```training/testing_set```.
    - Saved in ```word_lists/```.
        - Format : ```word_tag\n```
            - ex : ```Let_VBP\n```
    - Based on this word list, the script will look for the appropriate word bin to use in ```word_bins/unique``` and extract the specified number of sentences per bin.
    - Then, the first word will be retagged correctly (based on the tag specified in the word list).
    - *Note* : This retagging process is aggresive, and we can only specify one correct tag for each word.
    - Words for the training set must be followed by the words for the testing set.
    - --num_test_bins (=n) is used to split the word list into training and testing. (bottom n : testing, rest : training)

- --train (optional) 
    - Filename of txt file containing training set.
    - Saved in ```training_set/```.
    - This is used when we want to create new testing sets for training sets that we made previously, while ensuring that our training and testing sets do not have common sentences. 
    (Training set will not be created if this argument is used)
    - It is recommended to use this option whenever training and testing sets will be made using the same word bins. 
    (ie do not try to make the training and testing sets in one go (without using --train) if they are going to be created from the same bins.)
    
## Other arguments
- --num_train_sents, -ntr
    - Number of training sentences per bin.
    - Defaults to 5.

- --num_test_bins, -nte
    - Number of testing word bins.
    - Defaults to 1.

- --train_extension, -tr_e
    - Custom extension for training set filename. 
    - Adds string to the end of filename.

- --test_extension, -te_e
    - Custom extension for testing set filename. 
    - Adds string to the end of filename.

## Output
- txt files of sentences starting with specified word (in ```word_bins/``` or ```word_bins/unique/```)
    - Output is saved in ```word_bins/``` or ```word_bins/unique/``` depending on whether the -u flag is used.
    - The filename is automatically formatted to be the word (by which we're creating the word bin).

        > ```file_name = "word_bins/" + word + args.extension + ".txt"```

        > ```file_name_unique = "word_bins/unique/" + word + args.extension + ".txt"```

## Example usage

In [20]:
!nohup python3 make_test_train.py --num_train_sents 5 --num_test_bins 5 --train_extension test --test_extension test --word_list word_lists/nnp_verb_list_test.txt

nohup: ignoring input and appending output to 'nohup.out'


## Example usage 2

In [23]:
# Make new testing bin based on training bin
!nohup python3 make_test_train.py --train training_set/test.txt --num_test_bins 1 --test_extension test_2 --word_list word_lists/nnp_verb_list_partition.txt

nohup: ignoring input and appending output to 'nohup.out'


# ```main_experiment.py``` : Run main experiment

```main_experiment.py``` finds the optimal tagger that tags imperative verbs accurately.

## Input
- --training_set
    - Training set (txt file in ```training_set/```)
- --testing_set
    - Testing set (txt file in ```testing_set/```)

*Note* : Both training and testing sets are "correctly" tagged.
    They can be outputs of ```make_test_train.py``` but note that ```make_test_train``` only "corrects" the first word, according to the word list.
  
## Other arguments
- --train, -tr
    - txt file to read training set.

- --test, -te
    - txt file to read testing set.

- --cores, -p
    - Number of cores to use. 
    - Defaults to 4.

- --extension, -e
    - Custom extension for filename. 
    - Adds string to the end of filename.

- --num_trials, -nt
    - Number of trials of experiment.
    - Defaults to 10.

- --train_num_lis, -tnl
    - Number of sentences to train on (list in string format).
    - Defaults to 5,10.
    
- --iter_num_list, -inl
    - Number of iterations (list in string format).
    - Defaults to 5,10.

- --wsj_test, -wt
    - Tests on WSJ testing corpus.

- --print_mislabels, -m
    - Prints non-VB tags.

- --dump, -d
    - Dumps trained tagger.

- --tag_n, -tn
    - Number of words to tag in each sentence (first n).
    - Defaults to None.

- --default_results, -dr
    - Writes default results to file.

## Output
- txt file of results.
    - Saved in ```experiments/```.
- Pickled taggers, depending on flags used.
    - Saved in ```tagger/```.

The filename is formatted automatically depending on the parameters
> ```output_test = "experiments/experiment_" + str(num_train_sent) + "sents_" + str(nr_iter) + "iters_" + extension + ".txt"```

If the -d flag is added, the taggers are dumped for future use (use ```dumped_main_experiment.py```)
The filename is formatted automatically (similar to the results, but with the trial ID added at the end)
>  ```output_dump = "tagger/" + str(num_train_sent) + "sents_" + str(nr_iter) + "iters_" + extension + "_" + str(trial_id) + ".pk"```

## Example usage

In [21]:
!nohup python3 main_experiment.py --train training_set/test.txt --test testing_set/test.txt --cores 30 --num_trials 5 --train_num_list 1,5 --iter_num_list 5 --extension test --wsj_test --dump --default_results 

nohup: ignoring input and appending output to 'nohup.out'


# ```read_experiment_results.py``` : Read experiment results

```read_experiment_results.py``` summarizes results from ```main_experiment.py``` or ```dumped_main_experiment.py``` by taking averages and standard deviations from multiple trials.

## Input
- --files, -f
    - txt files to read experiment results from.
    - Files should be outputs of ```main_experiment.py``` or ```dumped_main_experiment.py```.

## Output
- --output, -o 
    - Path of txt file of summarized results.

## How to read results
- Takes the results of n trials of an experiment (done under the same condition) in one txt file, and writes the average and standard deviation for each metric.
- Format :
    > ```file_name\taverage_of_first_metric,standard_deviation_of_first_metric\taverage_of_second_metric,standard_deviation_of_second_metric\t...\n``

- In the example the following metrics are reported, separated by ```\t```.
    - Accuracy
    - Number of VBs mislabeled as NNP
    - Number of VBGs mislabeled as NNP
    - Number of VBs mislabeled as NN
    - Number of NNs mislabeled as JJ
    - Number of NNs mislabeled as VB
    - Number of NNSs mislabeled as VBZ
    - Number of JJ mislabeled as NN
    - Number of RB mislabeled as NN
    - Number of VBs mislabeled
    - Number of mislabellings

## Example usage

In [22]:
!python3 read_experiment_results.py --files experiments/*test* --output experiments/summary_test.txt

experiment_1sents_5iters_test-wsj
[['0.97008534755474', '4', '4', '66', '164', '17', '32', '175', '22', '147', '3130'], ['0.9698368552341085', '5', '4', '70', '166', '16', '32', '171', '22', '156', '3156'], ['0.9698655274249506', '5', '4', '64', '175', '14', '31', '169', '23', '149', '3153'], ['0.9700375605700031', '3', '4', '64', '157', '12', '30', '175', '23', '146', '3135'], ['0.9703242824784242', '4', '4', '61', '164', '16', '24', '175', '25', '147', '3105']]
experiment_1sents_5iters_test
[['0.9461538461538461', '10', '0', '0', '3', '0', '0', '3', '0', '23', '42'], ['0.9564102564102565', '10', '0', '0', '7', '0', '1', '1', '0', '17', '34'], ['0.958974358974359', '9', '0', '0', '2', '0', '0', '2', '0', '16', '32'], ['0.9525641025641025', '12', '0', '0', '4', '0', '0', '3', '0', '20', '37'], ['0.9346153846153846', '16', '0', '0', '2', '0', '0', '4', '0', '30', '51']]
experiment_5sents_5iters_test-wsj
[['0.970372069463161', '4', '4', '63', '166', '14', '33', '177', '24', '141', '3100'

# ```dumped_main_experiment.py``` : Run dumped experiment

```dumped_main_experiment.py``` runs the main experiment on dumped taggers and return accuracy, # of mislabelled tags etc. (Refer to ```main_experiment.py``` for more detailed information.)


## Input
-  --tagger
    - Dumped taggers. 
    - Saved in ```tagger/```.
    - Dumped using ```main_experiment.py```.
- --test
    - Testing set.
    - Saved in ```testing_set/```. - Made using ```make_test_train.py```.
- --tag_n
    - Tags only first n words in each sentence.
    - Defaults to None, which tags all words.

*Note* : Both training and testing sets are "correctly" tagged.
    They can be outputs of ```make_test_train.py``` but note that ```make_test_train``` only "corrects" the first word, according to the word list.
  
## Other arguments
- --test, -te
    - txt file to read testing set from.
    
- --extension, -e
    - Custom extension for filename. 
    - Adds string to the end of filename.
    
- --cores, -p
    - Number of cores to use. 
    - Defaults to 4.

- --wsj_test, -wt
    - Tests on WSJ testing corpus.

- --print_mislabels, -m
    - Output non-VB tags.

- --tag_n, -tn
    - Number of words to tag in each sentence (first n).
    - Defaults to None.

## Output
- txt file of results.
    - Saved in ```experiments/```.
    - The filename is formatted automatically depending on the parameters.
        > ```output_test = "experiments/experiment_" + str(num_train_sent) + "sents_" + str(nr_iter) + "iters_" + extension + ".txt"```

## Example usage

In [24]:
!nohup python3 dumped_main_experiment.py --tagger tagger/5sents_5iters_test_trial1.pk --test testing_set/refer_handtagged.txt --extension refer_test -p 25 --tag_n 3

nohup: ignoring input and appending output to 'nohup.out'


# ```find_disagreeing_sents.py``` : Finds sentences with "disagreeing" first word tags.

```find_disagreeing_sents.py``` finds sentences with first words such that the best taggers agree on the tag (best tag), the worst taggers agree on the tag (worst tag), and best tag and worst tag are not the same.

## Input
- --file, -f
    - txt files of tagged sentences to find disagreeing sentences from.

- --best_tagger, -b
    - IDs of best taggers, split by commas.
    - Defaults to 41,9.

- --worst_tagger, -w
    - IDs of worst taggers, split by commas.
    - Defaults to 38,22.

*Note* : Taggers are loaded from :
> ```TAGGER_PATH = "tagger/7_5/5sents_5iters_7_5_trial"```

  
## Other arguments
- --use_default, -d
    - Uses default tagger.

- --write_tags, -t
    - Tag sentences using every tagger (best and worst) and write the tags out.
    - Filename is formatted as follows :
        > ```"trial" + all_taggers_ids[ind] + ".txt"```

## Output
- --output, -o
    - Path of txt file to write sentences to. 
    - If unspecified, results are printed.

## How to look at the output
>```best_tag_of_first_word\tworst_tag_of_first_word\tsent```

## Example usage

In [ ]:
!python3 find_disagreeing_sents.py -f testing_set/4verbs_handtagged.txt -o disagreeing_sents_7_8.txt

# ```compare_tagger_weights.py``` : Compare tagger weights

```compare_tagger_weights.py``` compares weights of trained taggers using output of ```find_disagreeing_sents.py```.

## Input
- --file, -f
    - txt file of sentences.
    - In ```disagreeing_sents/```, output of ```disagreeing_sents.py```.

- --best_tagger, -b
    - IDs of best taggers, split by commas.
    - Defaults to 41,9.

- --worst_tagger, -w
    - IDs of worst taggers, split by commas.
    - Defaults to 38,22.

*Note* : Taggers are loaded from :
> ```TAGGER_PATH = "tagger/7_5/5sents_5iters_7_5_trial"```

  
## Other arguments
- --use_default, -d
    - Uses default tagger.

## Output
- --output, -o
    - Path of txt file to write to. 
    - If unspecified, results are printed.

## Example usage

In [26]:
!python3 compare_tagger_weights.py --file disagreeing_sents/disagreeing_sents_7_8.txt --output compare_weights_test.txt

loaded
